In [4]:
import numpy as np
import pandas as pd
import random
eps = np.finfo(float).eps
from numpy import log2 as log

In [5]:
mid1 = []
mid2 = []
project= []
for i in range(10):
    mid1.append(random.randint(1,101))
    mid2.append(random.randint(1,101))
    project.append(random.randint(1,101))
data = {'Mid1' : mid1, 'Mid2':mid2, 'Project':project}
df_data = pd.DataFrame(data, columns=['Mid1', 'Mid2', 'Project'])
# print(df_data)
mid1q = []
mid2q = []
projectq = []
markings=[mid1q, mid2q, projectq]
markings[0] = list(df_data.Mid1.quantile([0.25,0.5,0.75]))
markings[1] = list(df_data.Mid2.quantile([0.25,0.5,0.75]))
markings[2] = list(df_data.Project.quantile([0.25,0.5,0.75]))
print(markings[0][0])
#     markings[i]['median'] = np.median(df_data.Mid1)
print(data['Mid1'][0])
for j in data:    
    count = 0
    for i in range(len(mid1)):
#         if markings[j][i] < markings[j]['quartiles']:
        if data[j][i] < markings[count][0]:
            data[j][i] = 'poor'
        elif data[j][i] >= markings[count][0] and data[j][i] < markings[count][1]:
            data[j][i] = 'poor'
        elif data[j][i] >= markings[count][1] and data[j][i] < markings[count][2]:
            data[j][i] = 'average'
        else:
            data[j][i] = 'Fine'
    count+=1
data

34.75
19


{'Mid1': ['poor',
  'Fine',
  'average',
  'Fine',
  'poor',
  'poor',
  'Fine',
  'average',
  'poor',
  'poor'],
 'Mid2': ['poor',
  'average',
  'poor',
  'poor',
  'poor',
  'poor',
  'Fine',
  'Fine',
  'average',
  'poor'],
 'Project': ['average',
  'poor',
  'average',
  'poor',
  'Fine',
  'poor',
  'poor',
  'average',
  'poor',
  'average']}

In [11]:

dataset = {'Mid1':data['Mid1'],
       'Mid2':data['Mid2'],
       'Project':data['Project'],
       'result':['No','Yes','No','No','No','No','Yes','Yes','No','No']}

In [12]:
df = pd.DataFrame(dataset,columns=['Mid1','Mid2','Project','result'])

In [13]:
df

,Mid1,Mid2,Project,result
0,poor,poor,average,No
1,Fine,average,poor,Yes
2,average,poor,average,No
3,Fine,poor,poor,No
4,poor,poor,Fine,No
5,poor,poor,poor,No
6,Fine,Fine,poor,Yes
7,average,Fine,average,Yes
8,poor,average,poor,No
9,poor,poor,average,No


In [14]:
entropy_node = 0  #Initialize Entropy
values = df.result.unique()  #Unique objects - 'Yes', 'No'
for value in values:
    fraction = df.result.value_counts()[value]/len(df.result)  
    entropy_node += -fraction*np.log2(fraction)

In [15]:

attribute = 'Project'
target_variables = df.result.unique()  #This gives all 'Yes' and 'No'
variables = df[attribute].unique()    #This gives different features in that attribute (like 'Sweet')
entropy_attribute = 0
for variable in variables:
    entropy_each_feature = 0
    for target_variable in target_variables:
        num = len(df[attribute][df[attribute]==variable][df.result ==target_variable]) #numerator
        den = len(df[attribute][df[attribute]==variable])  #denominator
        fraction = num/(den+eps)  #pi
        entropy_each_feature += -fraction*log(fraction+eps) #This calculates entropy for one feature like 'Sweet'
    fraction2 = den/len(df)
    entropy_attribute += -fraction2*entropy_each_feature   #Sums up all the entropy ETaste


In [16]:
abs(entropy_attribute)

0.8099865470109868

In [17]:

def find_entropy(df):
    Class = df.keys()[-1]   
    entropy = 0
    values = df[Class].unique()
    for value in values:
        fraction = df[Class].value_counts()[value]/len(df[Class])
        entropy += -fraction*np.log2(fraction)
    return entropy
  
def find_entropy_attribute(df,attribute):
    Class = df.keys()[-1]   
    target_variables = df[Class].unique()  #This gives all 'Yes' and 'No'
    variables = df[attribute].unique()    #This gives different features in that attribute (like 'Poor','Average')
    entropy2 = 0
    for variable in variables:
        entropy = 0
        for target_variable in target_variables:
            num = len(df[attribute][df[attribute]==variable][df[Class] ==target_variable])
            den = len(df[attribute][df[attribute]==variable])
            fraction = num/(den+eps)
            entropy += -fraction*log(fraction+eps)
        fraction2 = den/len(df)
        entropy2 += -fraction2*entropy
    return abs(entropy2)

In [18]:
def find_winner(df):
    Entropy_att = []
    IG = []
    for key in df.keys()[:-1]:
#         Entropy_att.append(find_entropy_attribute(df,key))
        IG.append(find_entropy(df)-find_entropy_attribute(df,key))
    return df.keys()[:-1][np.argmax(IG)]
  
def get_subtable(df, node,value):
    return df[df[node] == value].reset_index(drop=True)


In [19]:
def buildTree(df,tree=None): 
    Class = df.keys()[-1]   
    node = find_winner(df)
    attValue = np.unique(df[node])
    
    #Create an empty dictionary to create tree    
    if tree is None:                    
        tree={}
        tree[node] = {}
    
   #We make loop to construct a tree by calling this function recursively. 
    #In this we check if the subset is pure and stops if it is pure. 

    for value in attValue:
        
        subtable = get_subtable(df,node,value)
        clValue,counts = np.unique(subtable['result'],return_counts=True)                        
        
        if len(counts)==1:#Checking purity of subset
            tree[node][value] = clValue[0]                                                    
        else:        
            tree[node][value] = buildTree(subtable) #Calling the function recursively 
                   
    return tree

In [20]:
tree = buildTree(df)

In [21]:
import pprint

In [23]:
pprint.pprint(tree)

{'Mid2': {'Fine': 'Yes',
          'average': {'Mid1': {'Fine': 'Yes', 'poor': 'No'}},
          'poor': 'No'}}


In [24]:
def predict(inst,tree):
    #This function is used to predict for any input variable 
    
    #Recursively we go through the tree that we built earlier

    for nodes in tree.keys():        
        
        value = inst[nodes]
        tree = tree[nodes][value]
        prediction = 0
            
        if type(tree) is dict:
            prediction = predict(inst, tree)
        else:
            prediction = tree
            break;                            
        
    return prediction

In [25]:
inst = df.iloc[5]

In [26]:
inst

Mid1       poor
Mid2       poor
Project    poor
result       No
Name: 5, dtype: object

In [27]:
prediction = predict(inst, tree)

In [28]:
prediction


'No'

In [29]:
inst = df.iloc[4]

In [30]:
inst


Mid1       poor
Mid2       poor
Project    Fine
result       No
Name: 4, dtype: object